In [ ]:
import os
import pandas as pd
import cx_Oracle
from dotenv import load_dotenv

### Inicializálás

In [ ]:
load_dotenv()
username = os.getenv('ORACLE_USERNAME')
password = os.getenv('ORACLE_PASSWORD')
dsn = os.getenv('ORACLE_DSN')
pool = cx_Oracle.SessionPool(user=username, password=password, dsn=dsn, min=2, max=5, increment=1)

input_file = 'adószámok.xlsx'
output_file = 'elokeszit_adószámok.xlsx'

AHT='397451'
KID=f'{AHT}/2025'
SHEET='Munka1'

def get_data(ksh):
    query = f'''
select 
    onk.ksh, 
    SUBSTR(st.torzs_sz||'-'||st.afakod||'-'||st.mk, 0, 8) as torzsszam,    
    cim.irszam, 
    onk.onev,
    cim.koztnev||' '||cim.kozjell as cim,                       
    cim.hsz1 as hsz, 
    TRIM(rk.megnevezes) as regio
from SUPPORTSYS.t_onkorm_tech onk
left join (select * from SUPPORTSYS.t_stat) st on st.ksh=onk.ksh
left join (select * from SUPPORTSYS.t_cim) cim on cim.ksh=onk.ksh
left join SUPPORTSYS.t_regio rk on rk.regiokod=onk.rk
where onk.ksh='{ksh}'
'''
    connection = pool.acquire()
    cursor = connection.cursor()
    cursor.execute(query)
    data = cursor.fetchall()    
    pool.release(connection)
    return data

### EBR betöltő beolvasás

In [ ]:
df = pd.read_excel(f'./data/{input_file}', sheet_name=0, usecols=[0, 1], names=['KSH', 'ONK'], skiprows=0) #, nrows=41)
df['KSH'] = df['KSH'].astype(str).str.replace('\.0', '', regex=True).str.zfill(7)
# df['KSH'] = df['KSH'].astype(str).str.zfill(7)
# df['AMOUNT'] = df['AMOUNT'].fillna(0).astype(int)
df['KID'] = KID
df['TORZSSZAM'] = ''
df['AHT'] = AHT
df['IRSZAM'] = ''
df['ONEV'] = ''
df['CIM'] = ''
df['HSZ'] = ''
df['REGIO'] = ''
if df.iloc[-1]['KSH'] == '9999999':
  df = df.iloc[:-1]

df

### Szükséges adatok az adatbázisból

In [ ]:
column_names = ['TORZSSZAM', 'IRSZAM', 'ONEV', 'CIM', 'HSZ', 'REGIO']

for index, row in df.iterrows():
    ksh_value = row['KSH']
    data = get_data(ksh_value)
    for i, column_name in enumerate(column_names):
        df.at[index, column_name] = data[0][i+1]
df.head()

### Bezárás

In [ ]:
pool.close()
df.to_excel(f'./data/{output_file}', index=False)